<a href="https://colab.research.google.com/github/francesco-source/CDMO/blob/SAT/SAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from itertools import combinations
from z3 import *
import math
import numpy as np

In [37]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one(bool_vars, name):
    return And(at_least_one(bool_vars), at_most_one(bool_vars, name))



In [38]:
m = 3
n = 7
l = [15, 10, 20]
s = [3, 2, 6, 8, 5, 4, 4]
D = [[0, 3, 3, 6, 5, 6, 6, 2],
     [3, 0, 4, 3, 4, 7, 7, 3],
     [3, 4, 0, 7, 6, 3, 5, 3],
     [6, 3, 7, 0, 3, 6, 6, 4],
     [5, 4, 6, 3, 0, 3, 3, 3],
     [6, 7, 3, 6, 3, 0, 2, 4],
     [6, 7, 5, 6, 3, 2, 0, 4],
     [2, 3, 3, 4, 3, 4, 4, 0]]



In [39]:
class Postmans:
    def __init__(self, m, n, l, s, D):
        self.m = m
        self.n = n
        self.l = l
        self.s = s
        self.D = D

        
instance = Postmans(m,n,l,s,D)

In [40]:
#print function
def print_function(x,c,last,D):
    travel = {}
    for i in range(m):
        t = []
        for j in range(n):
            for k in range(0,n+1):
                if x[i][j][k] == True:
                    t.append(k)
        travel[i] = t                 
    print(travel)
            
    distance = {}
    total_distance = 0
    for i in range(m):
        d = 0
        for j in range(n):
            if j == 0:
                for k in range(1,n+1):
                    if x[i][j][k] == True:
                        d += D[n][k-1]
                        total_distance += D[n][k-1]
            for k in range(n):
                if c[i][j][k] == True:
                    d += D[j][k]
                    total_distance += D[j][k]
        for k in range(n):
            if last[i][k] == True:
                d += D[k][n]
                total_distance += D[k][n]
                
        distance[i] = d
    print(distance)
    print(f"total distance = {total_distance}\n")
    print("-------------------------------------")
    return total_distance
    

In [41]:
def mtps(instance):
      m = instance.m
      n = instance.n
      l = instance.l
      s = instance.s
      D = instance.D

      solver = Solver()

      constraints = []

      X = [[[Bool(f"x_{i}_{j}_{k}") for k in range(0,n+1)] for j in range(n)] for i in range(m)]
      C = [[[Bool(f"c_{i}_{j}_{k}") for k in range(n)] for j in range(n)] for i in range(m)]
      last = [[Bool(f"l_{i}_{j}") for j in range(n)] for i in range(m)]
      
      for i in range(m):
            solver.add(at_most_one(last[i], f"l_{i}"))
            

      upper_distance = int(np.sum(D)/n)

      #each cell has only one value.
      for i in range(m):
            for j in range(n):
                  solver.add(exactly_one(X[i][j],f"valid_cell_{i}_{j}"))

      #each element except zero should be seen only once inside the matrix
      for k in range(1,n+1):
            solver.add(exactly_one([X[i][j][k] for i in range(m) for j in range(n)],f"valid_k{k}"))

      #each courier can only deliver items whose total size doesn't exceed limit
      for i in range(m):
            solver.add(PbLe([(X[i][j][k],s[k-1]) for j in range(n) for k in range(1,n+1)], l[i]))
      
      #metti gli zero in fondo
      for i in range(m):
            for j in range(n-1):
                  solver.add(Implies(X[i][j][0],X[i][j+1][0]))

      #C serve per dire se il corriere va da un posto a un altro
      for i in range(m):
            for j in range(n-1):
                  for k in range(1,n+1):
                        for kk in range(1,n+1):
                              solver.add(Implies(X[i][j][k], Implies(X[i][j+1][kk], C[i][k-1][kk-1])))

      #ultimo posto visitato
      for i in range(m):
            for j in range(n-1):
                  for k in range(1,n+1):
                        #solver.add(Xor(Implies(And(X[i][j][k], X[i][j+1][0]),last[i][k-1]),Implies(X[i][n-1][k],last[i][k-1])))
                        solver.add(Implies(And(X[i][j][k], X[i][j+1][0]),last[i][k-1]))
      
      #distance
      solver.add(PbLe([(C[i][j][k], D[j][k]) for j in range(n) for k in range(n) for i in range(m)]
                 +[(X[i][0][k],D[n][k-1]) for k in range(1,n+1) for i in range(m)]
                 +[(last[i][j],D[j][n]) for j in range(n) for i in range(m)], upper_distance)) 
      
      
      total_distance = upper_distance

      while solver.check() == sat:
            model = solver.model()
            
            
            x = [[[ model.evaluate(X[i][j][k]) for k in range(0,n+1) ] for j in range(n) ] for i in range(m)]
            c = [[[ model.evaluate(C[i][j][k]) for k in range(n) ] for j in range(n) ] for i in range(m) ]
            la = [[ model.evaluate(last[i][j]) for j in range(n) ] for i in range(m) ]
            total_distance = print_function(x,c,la,D)
            
            
            solver.push()
            solver.add(PbLe([(C[i][j][k], D[j][k]) for j in range(n) for k in range(n) for i in range(m)]
                            +[(X[i][0][k],D[n][k-1]) for k in range(1,n+1) for i in range(m)]
                            +[(last[i][j],D[j][n]) for j in range(n) for i in range(m)], total_distance-1)) 
            
      '''      
      solver.pop()
      solver.check() == sat
      model = solver.model()
      x = [[[ model.evaluate(X[i][j][k]) for k in range(0,n+1) ] for j in range(n) ] for i in range(m)]
      c = [[[ model.evaluate(C[i][j][k]) for k in range(n) ] for j in range(n) ] for i in range(m) ]
      la = [[model.evaluate(last[i][j]) for j in range(n)] for i in range(m)]
      print_function(x,c,la,D)'''
      
      
mtps(instance)


{0: [1, 7, 5, 0, 0, 0, 0], 1: [4, 2, 0, 0, 0, 0, 0], 2: [3, 6, 0, 0, 0, 0, 0]}
{0: 14, 1: 10, 2: 10}
total distance = 34

-------------------------------------
{0: [3, 6, 0, 0, 0, 0, 0], 1: [5, 7, 0, 0, 0, 0, 0], 2: [1, 2, 4, 0, 0, 0, 0]}
{0: 10, 1: 10, 2: 12}
total distance = 32

-------------------------------------
{0: [4, 2, 0, 0, 0, 0, 0], 1: [1, 3, 0, 0, 0, 0, 0], 2: [6, 7, 5, 0, 0, 0, 0]}
{0: 10, 1: 8, 2: 12}
total distance = 30

-------------------------------------
{0: [4, 2, 0, 0, 0, 0, 0], 1: [1, 0, 0, 0, 0, 0, 0], 2: [5, 7, 6, 3, 0, 0, 0]}
{0: 10, 1: 4, 2: 14}
total distance = 28

-------------------------------------
{0: [1, 2, 4, 0, 0, 0, 0], 1: [0, 0, 0, 0, 0, 0, 0], 2: [5, 7, 6, 3, 0, 0, 0]}
{0: 12, 1: 0, 2: 14}
total distance = 26

-------------------------------------
